In [ ]:
#importing dependencies 
import pandas as pd
from pathlib import Path 
import seaborn as sns
import tensorflow as tf

In [ ]:
#importing medical_df
medical_df = pd.read_csv("resources/medical_df.csv")

In [ ]:
obesity_df = medical_df[['Obesity','Total Income per Individual',
'Lacking Health Insurance','Binge Drinking','High Blood Pressure',
'Routine Health Checkups','Currently Smoking', 'Dentist Visits', 
'Depression', 'High Cholesterol', 'No Leisure-Time Physical Activity', 
"Less than 7 Hours of Sleep" ]]

obesity_df

## Splitting Data into training sets 

In [ ]:
train_dataset = cancer_df.sample(frac=0.8, random_state=1)
test_dataset = cancer_df.drop(train_dataset.index)

In [ ]:
train_dataset.describe().transpose()

## Splitting features from labels

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Obesity')
test_labels = test_features.pop('Obesity'	)

In [ ]:
#Normalization of Data 
train_dataset.describe().transpose()[["mean","std"]]

In [ ]:
#Data Normalization 
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
#Fitting state of preprocessing layer to the data by calling Normalization.adapt
normalizer.adapt(np.array(train_features))

In [ ]:
#Calculating the mean and varience and store them in a layer, when layer is called, it returns the input data, with each feature indepen normalized 
print(normalizer.mean.numpy())

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress = True):
    print('First example:' first)
    print()
    print('Normalized: ' , normalizer(first).numpy())

## Linear Regression with multiple inputs 

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
# When Model.predict is call on batch of inputs, it produces units=1 outputs for each example 
linear_model.predict(train_features)[:10]

In [ ]:
#When model is called, its weighted matrices will be built-check that the kernal weights(the m in y = mx + b) have a shape of (9,1)
linear_model.layers[1].kernel

In [ ]:
#Configuring moel with Keras Model.compile and train with Model.fit for 100 epochs
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

## Regression with a deep neural network (DNN)


In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

## Regression using a DNN and multiple inputs

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)


In [ ]:
plot_loss(history)

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [Cancer Risk]']).T


In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ')
plt.ylabel('Predictions ')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
#Error Distribution
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error ')
plt.ylabel('Count')

In [ ]:
#dnn_model.save('dnn_model')

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T
